## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Latitude,Longitude,Max Temperature,Percent Humidity,Percent Cloudiness,Wind Speed,Weather Description
0,0,Kavieng,PG,-2.5744,150.7967,82.74,71,100,6.38,overcast clouds
1,1,East London,ZA,-33.0153,27.9116,69.82,57,0,6.80,clear sky
2,2,Geraldton,AU,-28.7667,114.6000,57.61,94,23,6.91,few clouds
3,3,Rikitea,PF,-23.1203,-134.9692,77.02,72,14,20.56,few clouds
4,4,Nikolskoye,RU,59.7035,30.7861,41.88,49,84,6.33,broken clouds


In [3]:
# Prompt the user to input desired maximum and minimum temperatures
max_temp = float(input("What is the maximum maximum temperature you wish for your destination? "))
min_temp = float(input("What is the mininum maximum temperature you wish for your destination? "))

What is the maximum maximum temperature you wish for your destination? 75
What is the mininum maximum temperature you wish for your destination? 45


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_df = city_data_df.loc[(city_data_df['Max Temperature'] > 45) & (city_data_df['Max Temperature'] < 75)]

In [5]:
# 4a. Determine if there are any empty rows.
preferred_df.isnull().sum()

City_ID                0
City                   0
Country                2
Latitude               0
Longitude              0
Max Temperature        0
Percent Humidity       0
Percent Cloudiness     0
Wind Speed             0
Weather Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_df = preferred_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
#Rename preferred_df columns
preferred_df = preferred_df.rename(columns={'Latitude': 'Lat', "Longitude": "Lng", "Max Temperature": "Max Temp",
                            "Weather Description": "Current Description"})
#Select and order columns and create hotel_df copy
hotel_df = preferred_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.reindex(range(len(hotel_df)))

hotel_df['Current Description'] = [row['Current Description'].title() for i, row in hotel_df.iterrows()]

hotel_df.tail()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
673,Stromness,GB,51.76,Scattered Clouds,58.9650,-3.2960,
677,Guangshui,CN,49.44,Overcast Clouds,31.6199,113.9978,
679,Camacha,PT,63.82,Scattered Clouds,33.0833,-16.3333,
681,Killybegs,IE,56.91,Scattered Clouds,54.6333,-8.4500,
684,Laguna,US,58.01,Clear Sky,38.4210,-121.4238,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for i, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    Lat = row['Lat']
    Lng = row['Lng']
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    params['location'] = f"{Lat},{Lng}"
    # 6e. Make request and retrieve the JSON data from the search. 
    response = requests.get(url, params=params).json()
    hotel_results = response['results']
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    city = hotel_df.loc[i, 'City']
    try:
        hotel_df.loc[i, 'Hotel Name'] = hotel_results[0]['name']
    except:
        hotel_df.loc[i, 'Hotel Name'] = np.nan
        print(f"Missing {city} Hotel info, skipping... ")
        pass

Missing Albany Hotel info, skipping... 
Missing Kirovsk Hotel info, skipping... 
Missing Airai Hotel info, skipping... 
Missing Camacupa Hotel info, skipping... 
Missing Takapau Hotel info, skipping... 
Missing Yumen Hotel info, skipping... 
Missing Mus Hotel info, skipping... 
Missing Dergachi Hotel info, skipping... 
Missing Poum Hotel info, skipping... 
Missing Kasongo-Lunda Hotel info, skipping... 
Missing Chala Hotel info, skipping... 
Missing Poltavka Hotel info, skipping... 
Missing Shubarshi Hotel info, skipping... 


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Lodging</dt><dd>{Hotel Name}</dd>
<dt>Name</dt><dd>{City}<dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>High</dt><dd>{Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# 11b. Display the figfigure
fig

Figure(layout=FigureLayout(height='420px'))